In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-01-19 08:49:17--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  1.95MB/s    in 0.5s    

2025-01-19 08:49:18 (1.95 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt','r',encoding='utf-8') as f:
    text = f.read()

print("length of dataset in chars: ", len(text))

print(text[:1000])

In [4]:
# these are all unique chars
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
for i,ch in enumerate(chars):
    print(i, ch)

0 

1  
2 !
3 $
4 &
5 '
6 ,
7 -
8 .
9 3
10 :
11 ;
12 ?
13 A
14 B
15 C
16 D
17 E
18 F
19 G
20 H
21 I
22 J
23 K
24 L
25 M
26 N
27 O
28 P
29 Q
30 R
31 S
32 T
33 U
34 V
35 W
36 X
37 Y
38 Z
39 a
40 b
41 c
42 d
43 e
44 f
45 g
46 h
47 i
48 j
49 k
50 l
51 m
52 n
53 o
54 p
55 q
56 r
57 s
58 t
59 u
60 v
61 w
62 x
63 y
64 z


In [34]:
# tokenize 
stoi = {ch:i for i,ch in enumerate(chars) }
itos = {i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[i] for i in l)

In [12]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [14]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [19]:
# feed batches
torch.manual_seed(1337)
batch_size = 4
block_size = 8
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb,yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('---')
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"when input is {context} the target is {target}")

# we train on the entire block, get transformer network used to seeing
# things from 1 to block_size
# this is useful later in inference

# transformer knows how to predict next character
# transformer will never recieve more than block_Size inputs
# when predicting next character


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---
when input is tensor([24]) the target is 43
when input is tensor([24, 43]) the target is 58
when input is tensor([24, 43, 58]) the target is 5
when input is tensor([24, 43, 58,  5]) the target is 57
when input is tensor([24, 43, 58,  5, 57]) the target is 1
when input is tensor([24, 43, 58,  5, 57,  1]) the target is 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target is 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target is 39
when input is tensor([44]) the target is 53
when input is tensor([44, 53]) the target is 56
when input is tensor([44, 53, 56]) the target is 

In [32]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits from the next token from a lookup table
        # nn.Embedding is 65x65
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        print(self.token_embedding_table)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integer
        # batch, time, channel (4, 8, 65)
        logits = self.token_embedding_table(idx) # (B,T,C)
        if targets is None:
            loss = None
        else: 
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            # pytorch cross_entropy funct expects B,C,T
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx) # calls forward
            # focus only on the last time step
            logits = logits[:,-1,:] # becomes (B,C)
            # apply softmax to get new probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) #(B,T+1)
        return idx

    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# expected loss would be -ln(1/65)


Embedding(65, 65)
torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [ ]:
idx = torch.zeros((1,1), dtype=torch.long) # batch 1, t = 1
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

In [42]:
# let's train the model now
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

tensor([1.], device='mps:0')


In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


In [163]:
# copy over code
batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
eval_iters = 200
learning_rate = 3e-4
device = 'cpu'
n_embed = 384
n_heads = 6
n_layer = 6
dropout = 0.2

if torch.backends.mps.is_available():
    device = torch.device("mps")
    x = torch.ones(1, device=device)
    print (x)
else:
    device = 'cpu'
    print ("MPS device not found.")

with open('input.txt','r',encoding='utf-8') as f:
    text = f.read()

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

#data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

stoi = {ch:i for i,ch in enumerate(chars) }
itos = {i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[i] for i in l)


tensor([1.], device='mps:0')


In [170]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
    
class Head(nn.Module):
    """ one head of self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape # (B,T,C)
        # T is block_size
        # C is head_size
        k = self.key(x) # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("Affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B,T,T)
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf')) # mask out (B,T,T)
        wei = F.softmax(wei, dim=-1) # (B,T,T) compute softmax to get probs
        wei = self.dropout(wei)
        # perform the weighted aggregation of values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B,T,T) @ (B,T,C) -> (B,T,C)
        return out
    
class MultiHeadAttention(nn.Module):
    """ multi-head attention in parallel """
    def __init__(self, num_heads, head_size):
        super().__init__() 
        # (B,T,C)
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B,T,C)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, n_embed*4),
            nn.ReLU(),
            nn.Linear(4*n_embed, n_embed),
            nn.Dropout(dropout)
            )

    ## FF should take (B,T,C) and output (B,T,C)
    def forward(self, x): # all tokens do this individually
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self, n_embed, n_heads):
        super().__init__()
        # fill ins
        # block is basically self-attention, followed by linear
        # with communication between 
        head_size = n_embed // n_heads
        self.sa_head = MultiHeadAttention(n_heads, head_size)
        self.ffnw = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed) # LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed) # LayerNorm(n_embed)
    
    def forward(self, x):
        # output of self attention is passed to linear output
        x = x + self.sa_head(self.ln1(x))
        x = x + self.ffnw(self.ln2(x))
        return x
    
class LayerNorm(nn.Module):
    def __init__(self, dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        #parameters (trained with backprop)
        self.gamma = nn.Parameter(torch.ones(dim, device=device))
        self.beta = nn.Parameter(torch.zeros(dim, device=device))

    def __call__(self, x):
        # calculate the forward pass
        xmean = x.mean(1, keepdim=True) # batch norm is just across columns (0)
        xvar = x.var(1, keepdim=True) # batch norm is just across columns (0)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) #normalize to unit variance
        return self.gamma * xhat + self.beta
        

# Model 
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits from the next token from a lookup table
        # positional embedding
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        # nn.Embedding is vocab_size x n_embed
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        # fill in self.block here
        self.blocks = nn.Sequential(
            *[Block(n_embed, n_heads) for l in range(n_layer)],
        )
        self.ln_final = nn.LayerNorm(n_embed) # LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integer
        B,T = idx.shape
        # batch, time, channel (4, 8, vocab_size)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        token_emb = self.token_embedding_table(idx) # (B,T,n_embed)
        x = token_emb + pos_emb # now holds positions and embeddings
        x = self.blocks(x) # compute 3 blocks
        x = self.ln_final(x)
        logits = self.lm_head(x) # (B,T,vocab_size)
        if targets is None:
            loss = None
        else: 
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            # pytorch cross_entropy funct expects B,C,T
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond) # calls forward
            # focus only on the last time step
            logits = logits[:,-1,:] / temperature# becomes (B,C)
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to get new probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) #(B,T+1)
        return idx

model = BigramLanguageModel().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
#


In [168]:
from tqdm import tqdm

for iter in tqdm(range(max_iters), colour='#00ff00'):
    # sample a batch of data
    xb,yb = get_batch('train')

    if iter % eval_interval == 0 and iter > 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # evaluate the loss
    logits, loss = model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


  2%|▎         | 100/4000 [00:27<17:33,  3.70it/s]

step 100: train loss 2.4719, val loss 2.4831


  5%|▌         | 200/4000 [01:22<17:33,  3.61it/s]  

step 200: train loss 2.4193, val loss 2.4454


  8%|▊         | 300/4000 [02:18<16:48,  3.67it/s]  

...

 22%|██▎       | 900/4000 [08:33<15:11,  3.40it/s]  

step 900: train loss 1.6741, val loss 1.8419


 25%|██▌       | 1000/4000 [09:33<14:34,  3.43it/s] 

step 1000: train loss 1.6192, val loss 1.7894


 28%|██▊       | 1100/4000 [10:34<14:00,  3.45it/s]  

In [167]:
# generate from the model
idx = torch.zeros((1,1), dtype=torch.long, device=device) # batch 1, t = 1
print(decode(model.generate(idx, max_new_tokens=400)[0].tolist()))


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [70]:
# consider the toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.rand(B,T,C)
x.shape

# what is the easiest way to communicate with past tokens?
# take the channels from past steps and summarize or average them
# this is extremely weak form of interaction, but at least you consider it
# for ever single batch element (for ever T-th token), calculate hte average
# of all the vectors in all the previous tokens and this token 

# bow = bag of words
# we want x[b,t] = mean_{<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

x[0]

tensor([[0.0783, 0.4956],
        [0.6231, 0.4224],
        [0.2004, 0.0287],
        [0.5851, 0.6967],
        [0.1761, 0.2595],
        [0.7086, 0.5809],
        [0.0574, 0.7669],
        [0.8778, 0.2434]])

In [71]:
xbow[0]

tensor([[0.0783, 0.4956],
        [0.3507, 0.4590],
        [0.3006, 0.3156],
        [0.3717, 0.4108],
        [0.3326, 0.3806],
        [0.3953, 0.4140],
        [0.3470, 0.4644],
        [0.4134, 0.4368]])

In [73]:
torch.tril(torch.ones(3,3))
# returns a triangle

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [77]:
# this is inefficient, we can be much more efficient with matmul and tril
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3)) # does a sum of variable number of rows, deposited into C
a = a / torch.sum(a, 1, keepdim=True) # now rows sum to 1, now when we do a @ b, get the avg
# equivalent to divide by 1,2,3,4 etc
b = torch.randint(0,10,(3,2)).float()
c = a @ b 
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)
print('--')

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
--


In [81]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T,T) @ (B,T,C) --> pytorch will create a (B,T,T) @ (B,T,C) --> (B,T,C)
xbow[0], xbow2[0]

(tensor([[0.0783, 0.4956],
         [0.3507, 0.4590],
         [0.3006, 0.3156],
         [0.3717, 0.4108],
         [0.3326, 0.3806],
         [0.3953, 0.4140],
         [0.3470, 0.4644],
         [0.4134, 0.4368]]),
 tensor([[0.0783, 0.4956],
         [0.3507, 0.4590],
         [0.3006, 0.3156],
         [0.3717, 0.4108],
         [0.3326, 0.3806],
         [0.3953, 0.4140],
         [0.3470, 0.4644],
         [0.4134, 0.4368]]))

In [88]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T) # affinities between tokens are data dependent
wei = wei.masked_fill(tril == 0, float('-inf')) # neg infinite means can't look forward
print(wei)# 
wei = F.softmax(wei, dim=-1)
print(wei)
xbow3 = wei @ x

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


In [99]:
# self attention toy problem
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.rand(B,T,C)

# self attention - every single token emits a query and key
# query vector = roughly is what am i looking for
# key vector = is roughly what do i contain
# we do a dot product between query and key
# that dot product becomes wei
# if key and queyr are aligned they will interact to a high amount
# and you get to learn more about this token 

# let's see a single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)      # (B,T,16)
q = query(x)    # (B,T,16)
# all tokens in all positions in BxT arrangement in parallel, independnetly produce
# a key and a query

# communicate
wei = q @ k.transpose(-2,-1)  # (B,T,16) x (B,16,T) -> (B,T,T)
# this is a t^2 matrix giving us the affinities now


tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # neg infinite means can't look forward
wei = F.softmax(wei, dim=-1) # through softmax will end up aggregating info
v = value(x)
out = wei @ v # v is tensor we aggregate
out.shape

torch.Size([4, 8, 16])

* Attention is a communication mechanism
a communication mechniasm where you have an umber of nodes in a dag, where you have edges pointed between nodes

every node has some vector of informaiton and aggregates via a weighted sum thorugh all nodes that point to it

done in a data dpeendnet manner

the first node is only pointed to by itself,
second node is pointed to its node and itself
etc. 

this is the structure of our dag in autoregressive scenario like language modeling

can be pointed through any arbitrary directed grah

* there is no notion of space. attention acts across a set of vecotrs in this graph. they have no diea where they are in a space
this is why we encode them positionally so that they know where they are.

this is different from convolition because convolution has a specific layout and convoluation filters act in sapce and its not like attnetion. 

* Exach example across batch dimension is of course processed completely indepndently and never "talk" to each other

* in an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. e.g. if you're doing sentiment analysis, you may want to have all tokens talk to each other fully. it's ok for this to talk to each other. 

* in decoder blocks you should keep things masked

* what is cross attention vs self-attention? self-attention all come from th same source (x), x produces, keys, queries and values. but attention is much more general than that. in encoder-decoder, you can have queries produced from x, but keys and values can come from a diffent source (e.g. a context you want to condition on), nodes on the side. then you read off information from the side. this is cross-attention. a separate soruce of nodes you want to pull informatio from into our ndoes.

* Attention(Q,K,V) = softmax(QK.T/sqrt(head_size))V. Why are they scaling attention by the sqrt of head_size? if you have unit gaussian inputs, K and Q are unit gaussian, you'll see wei.var() will be on the order of headsize, but if you divide by headsize, then the variance of wei will be 1. this is important because wei feeds into softamx. it's important for wei to be fairly diffuse. because of softmax, if wei takes on very negative or very positive, wei will converge to one hot numbers. This will make softamax way too peaky, then every node aggregates information from just one node. 

